# Imports

In [1]:
import sys

import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../../src")
import timeseries.datasets

from IPython.display import display, HTML

# Helper Functions

## transform_data

In [2]:
def transform_data(df, reported_count_row, data_start_row, get_index):
    new_data = {}

    for column in df.columns:
        series = df[column]
        reported_count = int(series.iloc[reported_count_row])
        actual_count = series.iloc[data_start_row:].notna().sum()
        
        if reported_count != actual_count:
            print(
                f"Error on column {column}: "
                f"reported count ({reported_count}) "
                f"does not match actual count ({actual_count})"
            )
        
        values = series.values[data_start_row : data_start_row + actual_count]
        index = get_index(series, len(values))
        new_series = pd.Series(values, index=index)

        new_data[column] = new_series
    
    return pd.DataFrame(new_data)


## transform_yearly_data

In [3]:
def transform_yearly_data(df):
    def get_index(series, count):
        year = int(series.iloc[1])
        start_date = f"{year}"
        return pd.period_range(start=start_date, periods=count, freq="Y")
        
    return transform_data(
        df, reported_count_row=0, data_start_row=2, get_index=get_index
    )

## transform_quarterly_data

In [4]:
def transform_quarterly_data(df):
    def get_index(series, count):
        year = int(series.iloc[1])
        quarter = int(series.iloc[2])
        start_date = f"{year}Q{quarter}"
        return pd.period_range(start=start_date, periods=count, freq="Q")
        
    return transform_data(
        df, reported_count_row=0, data_start_row=3, get_index=get_index
    )

## transform_monthly_data

In [5]:
def transform_monthly_data(df):
    def get_index(series, count):
        year = int(series.iloc[1])
        month = int(series.iloc[2])
        start_date = f"{year}-{month:02d}"
        return pd.period_range(start=start_date, periods=count, freq="M")
        
    return transform_data(
        df, reported_count_row=0, data_start_row=3, get_index=get_index
    )

## is_oos_align

In [6]:
def is_oos_align(is_df, oos_df):
    res_df = pd.DataFrame({
        column: [is_df[column].last_valid_index(), oos_df[column].first_valid_index()]
        for column in is_df.columns
    })
    
    aligned_mask = res_df.iloc[0] + 1 == res_df.iloc[1]
    is_aligned = np.all(aligned_mask)
    
    message = f"In Sample and Out of Sample align? {is_aligned}"
    color = "green" if is_aligned else "red"
    
    display(HTML(f'<span style="color:{color}">{message}</span>'))
    with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
        display(res_df)
    if not is_aligned:
        misaligned_cols = res_df.columns[~aligned_mask]
        with pd.option_context("display.max_rows", 10, "display.max_columns", None):
            display(res_df[misaligned_cols])

# Get the Dataset Manager

In [7]:
tourism = timeseries.datasets.tourism

# Yearly

## Raw Yearly Data

* The first row is the number of samples (years).
* The second row is the staring year.

In [8]:
in_df = tourism.load_yearly_in_data(verbose=False)
oos_df = tourism.load_yearly_oos_data(verbose=False)
with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(
        HTML(f"<strong>In Sample</strong>"), in_df,
        HTML(f"<strong>Out of Sample</strong>"), oos_df
    )
    

,Y1,Y2,Y3,Y4,Y5,...,Y514,Y515,Y516,Y517,Y518
0,11.0000,11.00,31.0,31.0,31.0,...,16.0,16.0,16.0,16.0,16.0
1,1979.0000,1979.00,1973.0,1973.0,1973.0,...,1981.0,1981.0,1981.0,1981.0,1981.0
2,25092.2284,887896.51,227921.0,18441.0,27186.0,...,1476.0,1121.0,1531.0,1389.0,1122.0
3,24271.5134,887068.98,230995.0,21934.0,27921.0,...,1127.0,1118.0,1397.0,1416.0,1601.0
4,25828.9883,971549.04,183635.0,23739.0,32385.0,...,893.0,1549.0,1134.0,1573.0,1553.0
...,...,...,...,...,...,...,...,...,...,...,...
342,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
343,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
344,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
345,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


,Y1,Y2,Y3,Y4,Y5,...,Y514,Y515,Y516,Y517,Y518
0,4.0000,4.00,4,4,4,...,4,4,4,4,4
1,1990.0000,1990.00,2004,2004,2004,...,1997,1997,1997,1997,1997
2,36555.6156,1371337.14,1051696,273925,169661,...,1654,1512,1221,1189,1111
3,37385.6371,1370958.35,1143089,308842,204625,...,1912,1505,1258,1322,1313
4,38431.9699,1473846.26,1159025,335697,213837,...,1679,1751,1173,1142,1536
5,40345.3300,1578269.43,1230927,395167,234763,...,1985,1269,1655,1562,1739


## Transform Yearly Data

In [9]:
in_transformed_df = transform_yearly_data(in_df)
oos_transformed_df = transform_yearly_data(oos_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(
        HTML(f"<strong>In Sample</strong>"), in_transformed_df,
        HTML(f"<strong>Out of Sample</strong>"), oos_transformed_df
    )

Error on column Y248: reported count (12) does not match actual count (13)
Error on column Y249: reported count (12) does not match actual count (13)
Error on column Y250: reported count (12) does not match actual count (13)
Error on column Y251: reported count (12) does not match actual count (13)
Error on column Y252: reported count (12) does not match actual count (13)
Error on column Y253: reported count (12) does not match actual count (13)
Error on column Y254: reported count (12) does not match actual count (13)
Error on column Y255: reported count (12) does not match actual count (13)
Error on column Y256: reported count (12) does not match actual count (13)
Error on column Y257: reported count (12) does not match actual count (13)
Error on column Y258: reported count (12) does not match actual count (13)
Error on column Y259: reported count (12) does not match actual count (13)
Error on column Y260: reported count (12) does not match actual count (13)
Error on column Y261: rep

,Y1,Y2,Y3,Y4,Y5,...,Y514,Y515,Y516,Y517,Y518
1960,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1961,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1962,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1963,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1964,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1999,NaN,NaN,802705.0,210899.0,172166.0,...,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,966008.0,253095.0,193837.0,...,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,935717.0,249707.0,173359.0,...,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,1000844.0,264967.0,172654.0,...,NaN,NaN,NaN,NaN,NaN


,Y1,Y2,Y3,Y4,Y5,...,Y514,Y515,Y516,Y517,Y518
1990,36555.6156,1371337.14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1991,37385.6371,1370958.35,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1992,38431.9699,1473846.26,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1993,40345.3300,1578269.43,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1994,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2003,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2004,NaN,NaN,1051696.0,273925.0,169661.0,...,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,1143089.0,308842.0,204625.0,...,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,1159025.0,335697.0,213837.0,...,NaN,NaN,NaN,NaN,NaN


## Combine Yearly Data

In [10]:
is_oos_align(in_transformed_df, oos_transformed_df)

all_dates = pd.period_range(
    start=min(in_transformed_df.index.min(), oos_transformed_df.index.min()),
    end=max(in_transformed_df.index.max(), oos_transformed_df.index.max()),
    freq="Y"
)
results_df = in_transformed_df.reindex(all_dates)
results_df.update(oos_transformed_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(results_df)

,Y1,Y2,Y3,Y4,Y5,...,Y514,Y515,Y516,Y517,Y518
0,1989,1989,2003,2003,2003,...,1996,1996,1996,1996,1996
1,1990,1990,2004,2004,2004,...,1997,1997,1997,1997,1997


,Y18
0,2003
1,2003


,Y1,Y2,Y3,Y4,Y5,...,Y514,Y515,Y516,Y517,Y518
1960,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1961,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1962,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1963,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1964,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2003,NaN,NaN,683791.0,186809.0,116966.0,...,NaN,NaN,NaN,NaN,NaN
2004,NaN,NaN,1051696.0,273925.0,169661.0,...,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,1143089.0,308842.0,204625.0,...,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,1159025.0,335697.0,213837.0,...,NaN,NaN,NaN,NaN,NaN


## Save Yearly Data

In [11]:
tourism.save_yearly_data(results_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(
        tourism.load_yearly_data(verbose=False)
    )

2025-10-07 15:13:53,970 - INFO - Saved DataFrame to processed directory: /Users/jagraves21/notebooks/n-beats/data/Tourism/processed/yearly.csv


,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18,Y19,Y20,Y21,Y22,Y23,Y24,Y25,Y26,Y27,Y28,Y29,Y30,Y31,Y32,Y33,Y34,Y35,Y36,Y37,Y38,Y39,Y40,Y41,Y42,Y43,Y44,Y45,Y46,Y47,Y48,Y49,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y67,Y68,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,Y90,Y91,Y92,Y93,Y94,Y95,Y96,Y97,Y98,Y99,Y100,Y101,Y102,Y103,Y104,Y105,Y106,Y107,Y108,Y109,Y110,Y111,Y112,Y113,Y114,Y115,Y116,Y117,Y118,Y119,Y120,Y121,Y122,Y123,Y124,Y125,Y126,Y127,Y128,Y129,Y130,Y131,Y132,Y133,Y134,Y135,Y136,Y137,Y138,Y139,Y140,Y141,Y142,Y143,Y144,Y145,Y146,Y147,Y148,Y149,Y150,Y151,Y152,Y153,Y154,Y155,Y156,Y157,Y158,Y159,Y160,Y161,Y162,Y163,Y164,Y165,Y166,Y167,Y168,Y169,Y170,Y171,Y172,Y173,Y174,Y175,Y176,Y177,Y178,Y179,Y180,Y181,Y182,Y183,Y184,Y185,Y186,Y187,Y188,Y189,Y190,Y191,Y192,Y193,Y194,Y195,Y196,Y197,Y198,Y199,Y200,Y201,Y202,Y203,Y204,Y205,Y206,Y207,Y208,Y209,Y210,Y211,Y212,Y213,Y214,Y215,Y216,Y217,Y218,Y219,Y220,Y221,Y222,Y223,Y224,Y225,Y226,Y227,Y228,Y229,Y230,Y231,Y232,Y233,Y234,Y235,Y236,Y237,Y238,Y239,Y240,Y241,Y242,Y243,Y244,Y245,Y246,Y247,Y248,Y249,Y250,Y251,Y252,Y253,Y254,Y255,Y256,Y257,Y258,Y259,Y260,Y261,Y262,Y263,Y264,Y265,Y266,Y267,Y268,Y269,Y270,Y271,Y272,Y273,Y274,Y275,Y276,Y277,Y278,Y279,Y280,Y281,Y282,Y283,Y284,Y285,Y286,Y287,Y288,Y289,Y290,Y291,Y292,Y293,Y294,Y295,Y296,Y297,Y298,Y299,Y300,Y301,Y302,Y303,Y304,Y305,Y306,Y307,Y308,Y309,Y310,Y311,Y312,Y313,Y314,Y315,Y316,Y317,Y318,Y319,Y320,Y321,Y322,Y323,Y324,Y325,Y326,Y327,Y328,Y329,Y330,Y331,Y332,Y333,Y334,Y335,Y336,Y337,Y338,Y339,Y340,Y341,Y342,Y343,Y344,Y345,Y346,Y347,Y348,Y349,Y350,Y351,Y352,Y353,Y354,Y355,Y356,Y357,Y358,Y359,Y360,Y361,Y362,Y363,Y364,Y365,Y366,Y367,Y368,Y369,Y370,Y371,Y372,Y373,Y374,Y375,Y376,Y377,Y378,Y379,Y380,Y381,Y382,Y383,Y384,Y385,Y386,Y387,Y388,Y389,Y390,Y391,Y392,Y393,Y394,Y395,Y396,Y397,Y398,Y399,Y400,Y401,Y402,Y403,Y404,Y405,Y406,Y407,Y408,Y409,Y410,Y411,Y412,Y413,Y414,Y415,Y416,Y417,Y418,Y419,Y420,Y421,Y422,Y423,Y424,Y425,Y426,Y427,Y428,Y429,Y430,Y431,Y432,Y433,Y434,Y435,Y436,Y437,Y438,Y439,Y440,Y441,Y442,Y443,Y444,Y445,Y446,Y447,Y448,Y449,Y450,Y451,Y452,Y453,Y454,Y455,Y456,Y457,Y458,Y459,Y460,Y461,Y462,Y463,Y464,Y465,Y466,Y467,Y468,Y469,Y470,Y471,Y472,Y473,Y474,Y475,Y476,Y477,Y478,Y479,Y480,Y481,Y482,Y483,Y484,Y485,Y486,Y487,Y488,Y489,Y490,Y491,Y492,Y493,Y494,Y495,Y496,Y497,Y498,Y499,Y500,Y501,Y502,Y503,Y504,Y505,Y506,Y507,Y508,Y509,Y510,Y511,Y512,Y513,Y514,Y515,Y516,Y517,Y518
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40671.0,103880.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Quarterly

## Raw Quarterly Data

* The first row is the number of samples (months).
* The second row is the staring year.
* The third row is the staring month.

In [12]:
in_df = tourism.load_quarterly_in_data(verbose=False)
oos_df = tourism.load_quarterly_oos_data(verbose=False)
with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(
        HTML(f"<strong>In Sample</strong>"), in_df,
        HTML(f"<strong>Out of Sample</strong>"), oos_df
    )

,q1,q2,q3,q4,q5,...,q423,q424,q425,q426,q427
0,55.0000,55.00,80.0,80.0,80.0,...,72.0,72.0,72.0,72.0,72.0
1,1979.0000,1979.00,1985.0,1985.0,1985.0,...,1981.0,1981.0,1981.0,1981.0,1981.0
2,1.0000,1.00,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0
3,3592.5500,172137.16,120977.0,23717.0,65370.0,...,4240.0,2564.0,17905.0,36220.0,13462.0
4,6409.3403,207720.03,162468.0,27405.0,73043.0,...,99575.0,39600.0,19490.0,110070.0,36300.0
...,...,...,...,...,...,...,...,...,...,...,...
120,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
122,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
123,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


,q1,q2,q3,q4,q5,...,q423,q424,q425,q426,q427
0,8.000,8.0000,8,8,8,...,8,8,8,8,8
1,1992.000,1992.0000,2005,2005,2005,...,1999,1999,1999,1999,1999
2,4.000,4.0000,1,1,1,...,1,1,1,1,1
3,6915.520,348666.6900,260754,74078,114837,...,3519,1509,30175,27015,6632
4,5696.010,289378.8300,293599,70931,131353,...,179870,73818,17003,81524,15820
...,...,...,...,...,...,...,...,...,...,...,...
6,17469.630,540909.5000,313820,95829,143360,...,28964,21905,19944,37638,8915
7,7672.665,386096.5000,264012,80207,127910,...,4093,2165,17156,29839,7435
8,6407.285,304618.0820,300244,76102,142113,...,181351,60775,18302,68792,18986
9,10330.300,417078.1932,276400,75129,139363,...,269379,75115,21966,83717,24239


## Transform Quarterly Data

In [13]:
in_transformed_df = transform_quarterly_data(in_df)
oos_transformed_df = transform_quarterly_data(oos_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(
        HTML(f"<strong>In Sample</strong>"), in_transformed_df,
        HTML(f"<strong>Out of Sample</strong>"), oos_transformed_df
    )

,q1,q2,q3,q4,q5,...,q423,q424,q425,q426,q427
1975Q1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975Q2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975Q3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975Q4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1976Q1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2004Q3,NaN,NaN,263708.0,63246.0,100165.0,...,NaN,NaN,NaN,NaN,NaN
2004Q4,NaN,NaN,305695.0,88251.0,122930.0,...,NaN,NaN,NaN,NaN,NaN
2005Q1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2005Q2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


,q1,q2,q3,q4,q5,...,q423,q424,q425,q426,q427
1992Q4,6915.520,348666.69,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1993Q1,5696.010,289378.83,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1993Q2,9507.025,361884.60,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1993Q3,17469.630,540909.50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1993Q4,7672.665,386096.50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2006Q3,NaN,NaN,276400.0,75129.0,139363.0,...,NaN,NaN,NaN,NaN,NaN
2006Q4,NaN,NaN,318369.0,104259.0,154547.0,...,NaN,NaN,NaN,NaN,NaN
2007Q1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007Q2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


## Combine Quarterly Data

In [14]:
is_oos_align(in_transformed_df, oos_transformed_df)

all_dates = pd.period_range(
    start=min(in_transformed_df.index.min(), oos_transformed_df.index.min()),
    end=max(in_transformed_df.index.max(), oos_transformed_df.index.max()),
    freq="M"
)
results_df = in_transformed_df.reindex(all_dates)
results_df.update(oos_transformed_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(results_df)

,q1,q2,q3,q4,q5,...,q423,q424,q425,q426,q427
0,1992Q3,1992Q3,2004Q4,2004Q4,2004Q4,...,1998Q4,1998Q4,1998Q4,1998Q4,1998Q4
1,1992Q4,1992Q4,2005Q1,2005Q1,2005Q1,...,1999Q1,1999Q1,1999Q1,1999Q1,1999Q1


,q1,q2,q3,q4,q5,...,q423,q424,q425,q426,q427
1975-03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975-04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975-06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1975-07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2007-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


## Save Quarterly Data

In [15]:
tourism.save_quarterly_data(results_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(
        tourism.load_quarterly_data(verbose=False)
    )

2025-10-07 15:13:55,035 - INFO - Saved DataFrame to processed directory: /Users/jagraves21/notebooks/n-beats/data/Tourism/processed/quarterly.csv


,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,q58,q59,q60,q61,q62,q63,q64,q65,q66,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,q86,q87,q88,q89,q90,q91,q92,q93,q94,q95,q96,q97,q98,q99,q100,q101,q102,q103,q104,q105,q106,q107,q108,q109,q110,q111,q112,q113,q114,q115,q116,q117,q118,q119,q120,q121,q122,q123,q124,q125,q126,q127,q128,q129,q130,q131,q132,q133,q134,q135,q136,q137,q138,q139,q140,q141,q142,q143,q144,q145,q146,q147,q148,q149,q150,q151,q152,q153,q154,q155,q156,q157,q158,q159,q160,q161,q162,q163,q164,q165,q166,q167,q168,q169,q170,q171,q172,q173,q174,q175,q176,q177,q178,q179,q180,q181,q182,q183,q184,q185,q186,q187,q188,q189,q190,q191,q192,q193,q194,q195,q196,q197,q198,q199,q200,q201,q202,q203,q204,q205,q206,q207,q208,q209,q210,q211,q212,q213,q214,q215,q216,q217,q218,q219,q220,q221,q222,q223,q224,q225,q226,q227,q228,q229,q230,q231,q232,q233,q234,q235,q236,q237,q238,q239,q240,q241,q242,q243,q244,q245,q246,q247,q248,q249,q250,q251,q252,q253,q254,q255,q256,q257,q258,q259,q260,q261,q262,q263,q264,q265,q266,q267,q268,q269,q270,q271,q272,q273,q274,q275,q276,q277,q278,q279,q280,q281,q282,q283,q284,q285,q286,q287,q288,q289,q290,q291,q292,q293,q294,q295,q296,q297,q298,q299,q300,q301,q302,q303,q304,q305,q306,q307,q308,q309,q310,q311,q312,q313,q314,q315,q316,q317,q318,q319,q320,q321,q322,q323,q324,q325,q326,q327,q328,q329,q330,q331,q332,q333,q334,q335,q336,q337,q338,q339,q340,q341,q342,q343,q344,q345,q346,q347,q348,q349,q350,q351,q352,q353,q354,q355,q356,q357,q358,q359,q360,q361,q362,q363,q364,q365,q366,q367,q368,q369,q370,q371,q372,q373,q374,q375,q376,q377,q378,q379,q380,q381,q382,q383,q384,q385,q386,q387,q388,q389,q390,q391,q392,q393,q394,q395,q396,q397,q398,q399,q400,q401,q402,q403,q404,q405,q406,q407,q408,q409,q410,q411,q412,q413,q414,q415,q416,q417,q418,q419,q420,q421,q422,q423,q424,q425,q426,q427
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Monthly

## Raw Monthly Data

* The first row is the number of samples (months).
* The second row is the staring year.
* The third row is the staring month.

In [16]:
in_df = tourism.load_monthly_in_data(verbose=False)
oos_df = tourism.load_monthly_oos_data(verbose=False)
with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(
        HTML(f"<strong>In Sample</strong>"), in_df,
        HTML(f"<strong>Out of Sample</strong>"), oos_df
    )

,m1,m2,m3,m4,m5,...,m362,m363,m364,m365,m366
0,163.0000,163.00,240.0,240.0,240.0,...,216.0,216.0,216.0,216.0,216.0
1,1979.0000,1979.00,1985.0,1985.0,1985.0,...,1981.0,1981.0,1981.0,1981.0,1981.0
2,1.0000,1.00,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0
3,1149.8700,65072.37,37047.0,7611.0,27693.0,...,901.0,798.0,3688.0,11464.0,3841.0
4,1053.8002,48612.20,32836.0,6859.0,15075.0,...,648.0,625.0,6638.0,9007.0,2768.0
...,...,...,...,...,...,...,...,...,...,...,...
307,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
308,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
309,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
310,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


,m1,m2,m3,m4,m5,...,m362,m363,m364,m365,m366
0,24.0000,24.0000,24,24,24,...,24,24,24,24,24
1,1992.0000,1992.0000,2005,2005,2005,...,1999,1999,1999,1999,1999
2,8.0000,8.0000,1,1,1,...,1,1,1,1,1
3,6611.1150,176215.2000,89162,25621,44171,...,1173,351,6267,8954,2719
4,4150.2395,160025.7000,66728,19333,28694,...,291,223,13068,6824,690
...,...,...,...,...,...,...,...,...,...,...,...
22,2466.3100,109861.6957,86872,25586,39952,...,87895,23586,7398,25726,7778
23,2851.7150,116636.7445,87775,21037,52498,...,84169,26294,5539,22754,7859
24,3671.8050,139426.5233,116104,33134,55218,...,29116,17851,7215,17023,4802
25,3806.7800,161014.9254,108333,36748,46420,...,1255,106,5842,9877,2426


## Transform Monthly Data

In [17]:
in_transformed_df = transform_monthly_data(in_df)
oos_transformed_df = transform_monthly_data(oos_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(
        HTML(f"<strong>In Sample</strong>"), in_transformed_df,
        HTML(f"<strong>Out of Sample</strong>"), oos_transformed_df
    )

,m1,m2,m3,m4,m5,...,m362,m363,m364,m365,m366
1979-01,1149.8700,65072.37,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-02,1053.8002,48612.20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-03,1388.8798,58452.59,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-04,1783.3702,57033.97,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-05,1921.0252,71498.95,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2005-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2005-06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2005-07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2005-08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


,m1,m2,m3,m4,m5,...,m362,m363,m364,m365,m366
1992-08,6611.1150,176215.20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1992-09,4150.2395,160025.70,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1992-10,2841.0000,150995.50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1992-11,1813.4400,112002.60,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1992-12,2261.0800,85668.59,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2007-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


## Combine Monthly Data

In [18]:
is_oos_align(in_transformed_df, oos_transformed_df)

all_dates = pd.period_range(
    start=min(in_transformed_df.index.min(), oos_transformed_df.index.min()),
    end=max(in_transformed_df.index.max(), oos_transformed_df.index.max()),
    freq="M"
)
results_df = in_transformed_df.reindex(all_dates)
results_df.update(oos_transformed_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", 10):
    display(results_df)

,m1,m2,m3,m4,m5,...,m362,m363,m364,m365,m366
0,1992-07,1992-07,2004-12,2004-12,2004-12,...,1998-12,1998-12,1998-12,1998-12,1998-12
1,1992-08,1992-08,2005-01,2005-01,2005-01,...,1999-01,1999-01,1999-01,1999-01,1999-01


,m1,m2,m3,m4,m5,...,m362,m363,m364,m365,m366
1979-01,1149.8700,65072.37,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-02,1053.8002,48612.20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-03,1388.8798,58452.59,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-04,1783.3702,57033.97,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1979-05,1921.0252,71498.95,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2007-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2007-08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


## Save Monthly Data

In [19]:
tourism.save_monthly_data(results_df)

with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(
        tourism.load_monthly_data(verbose=False)
    )

2025-10-07 15:13:56,097 - INFO - Saved DataFrame to processed directory: /Users/jagraves21/notebooks/n-beats/data/Tourism/processed/monthly.csv


,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,m16,m17,m18,m19,m20,m21,m22,m23,m24,m25,m26,m27,m28,m29,m30,m31,m32,m33,m34,m35,m36,m37,m38,m39,m40,m41,m42,m43,m44,m45,m46,m47,m48,m49,m50,m51,m52,m53,m54,m55,m56,m57,m58,m59,m60,m61,m62,m63,m64,m65,m66,m67,m68,m69,m70,m71,m72,m73,m74,m75,m76,m77,m78,m79,m80,m81,m82,m83,m84,m85,m86,m87,m88,m89,m90,m91,m92,m93,m94,m95,m96,m97,m98,m99,m100,m101,m102,m103,m104,m105,m106,m107,m108,m109,m110,m111,m112,m113,m114,m115,m116,m117,m118,m119,m120,m121,m122,m123,m124,m125,m126,m127,m128,m129,m130,m131,m132,m133,m134,m135,m136,m137,m138,m139,m140,m141,m142,m143,m144,m145,m146,m147,m148,m149,m150,m151,m152,m153,m154,m155,m156,m157,m158,m159,m160,m161,m162,m163,m164,m165,m166,m167,m168,m169,m170,m171,m172,m173,m174,m175,m176,m177,m178,m179,m180,m181,m182,m183,m184,m185,m186,m187,m188,m189,m190,m191,m192,m193,m194,m195,m196,m197,m198,m199,m200,m201,m202,m203,m204,m205,m206,m207,m208,m209,m210,m211,m212,m213,m214,m215,m216,m217,m218,m219,m220,m221,m222,m223,m224,m225,m226,m227,m228,m229,m230,m231,m232,m233,m234,m235,m236,m237,m238,m239,m240,m241,m242,m243,m244,m245,m246,m247,m248,m249,m250,m251,m252,m253,m254,m255,m256,m257,m258,m259,m260,m261,m262,m263,m264,m265,m266,m267,m268,m269,m270,m271,m272,m273,m274,m275,m276,m277,m278,m279,m280,m281,m282,m283,m284,m285,m286,m287,m288,m289,m290,m291,m292,m293,m294,m295,m296,m297,m298,m299,m300,m301,m302,m303,m304,m305,m306,m307,m308,m309,m310,m311,m312,m313,m314,m315,m316,m317,m318,m319,m320,m321,m322,m323,m324,m325,m326,m327,m328,m329,m330,m331,m332,m333,m334,m335,m336,m337,m338,m339,m340,m341,m342,m343,m344,m345,m346,m347,m348,m349,m350,m351,m352,m353,m354,m355,m356,m357,m358,m359,m360,m361,m362,m363,m364,m365,m366
0,1149.8700,65072.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1053.8002,48612.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN